In [ ]:
with open('text.txt','r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print(len(text))

296230


In [ ]:
print(text[:1000])

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡£¬¯°²³´¹àáåéíóøúüĀāġĪīłńśūɪɴʀʏʖʙʜʟʻʾʿ˹˺͜͟͠͡πσφω،؟ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىيًٌٍَُِّْ١٢٣٤٥٧٩ٰٱہیۗۚۡಠᴀᴅᴇᴋᴛᴜᴡᴢḤḥṬṭ ​‌‍‎‏–—‘’“”•…′‵‼⁠⁵₹⃣→↗≦≧⋈⌐╯╰▀▄█░▒■□▽○●◕◡☝☠☹♀♂♡♪⚠⚡✂✅✊✔✦✧✨✳❁❇❌❓❝❞➡⨝　っツノヮヾ皿ﷺﷻ﷽️︶（）＾～･ﾉﾟ￣🇦🇧🇨🇩🇪🇫🇬🇭🇮🇯🇰🇱🇲🇳🇴🇵🇷🇸🇹🇺🇻🇽🇾🇿🌀🌌🌚🌫🍥🍴🍽🎃🎉🎊🏡🏻🏼🏿🐘🐙🐟🐧🐸🐼👀👇👉👌👍👎👥👨👹👻👽👾💀💎💠💨💪💫💯💵📖📚📲🔥🔫😀😁😂😃😅😆😇😉😋😌😍😎😏😑😒😔😕😖😝😞😟😡😢😣😤😨😩😫😬😭😮😰😳😴😵😶🙁🙂🙃🙄🙊🙋🙌🚨🤌🤍🤓🤔🤖🤙🤡🤣🤦🤧🤨🤫🤬🤮🤯🤷🤺🤼🥱🥲🥴🥶🥸🥹🥺🦐🦞🦯🧐🧜🧠🧢🫠🫡🫣🫥
454


In [ ]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode(" Bismillah"))
print(decode(encode(" Bismillah")))

[1, 35, 74, 84, 78, 74, 77, 77, 66, 73]
 Bismillah


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([296230]) torch.int64
tensor([19, 16, 18, 23, 16, 19, 20, 13,  1, 18, 25, 27, 18, 25,  1, 14,  1, 46,
        70, 84, 84, 66, 72, 70, 84,  1, 66, 79, 69,  1, 68, 66, 77, 77, 84,  1,
        66, 83, 70,  1, 70, 79, 69, 14, 85, 80, 14, 70, 79, 69,  1, 70, 79, 68,
        83, 90, 81, 85, 70, 69, 15,  1, 47, 80,  1, 80, 79, 70,  1, 80, 86, 85,
        84, 74, 69, 70,  1, 80, 71,  1, 85, 73, 74, 84,  1, 68, 73, 66, 85, 13,
         1, 79, 80, 85,  1, 70, 87, 70, 79,  1])


In [ ]:
text[:100]

'2/16/23, 18:18 - Messages and calls are end-to-end encrypted. No one outside of this chat, not even '

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
print(train_data[:block_size])
print(train_data[1:block_size+1])

tensor([19, 16, 18, 23, 16, 19, 20, 13])
tensor([16, 18, 23, 16, 19, 20, 13,  1])


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(context,'=>',target)

tensor([19]) => tensor(16)
tensor([19, 16]) => tensor(18)
tensor([19, 16, 18]) => tensor(23)
tensor([19, 16, 18, 23]) => tensor(16)
tensor([19, 16, 18, 23, 16]) => tensor(19)
tensor([19, 16, 18, 23, 16, 19]) => tensor(20)
tensor([19, 16, 18, 23, 16, 19, 20]) => tensor(13)
tensor([19, 16, 18, 23, 16, 19, 20, 13]) => tensor(1)


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # ur avg batch size
block_size = 8 # context length

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension = 4
  for t in range(block_size): # time dimension = 8
    context = xb[b, :t+1]
    target = yb[b,t]
    print(context.tolist(),'=>',target.tolist())

inputs
torch.Size([4, 8])
tensor([[80, 69, 84, 13,  1, 85, 73, 70],
        [73, 66, 69,  1, 85, 80,  1, 77],
        [84, 70,  1, 66, 85, 86,  1, 73],
        [84, 73, 90, 69,  0, 67, 73, 66]])
targets
torch.Size([4, 8])
tensor([[69, 84, 13,  1, 85, 73, 70, 90],
        [66, 69,  1, 85, 80,  1, 77, 70],
        [70,  1, 66, 85, 86,  1, 73, 74],
        [73, 90, 69,  0, 67, 73, 66, 74]])
----
[80] => 69
[80, 69] => 84
[80, 69, 84] => 13
[80, 69, 84, 13] => 1
[80, 69, 84, 13, 1] => 85
[80, 69, 84, 13, 1, 85] => 73
[80, 69, 84, 13, 1, 85, 73] => 70
[80, 69, 84, 13, 1, 85, 73, 70] => 90
[73] => 66
[73, 66] => 69
[73, 66, 69] => 1
[73, 66, 69, 1] => 85
[73, 66, 69, 1, 85] => 80
[73, 66, 69, 1, 85, 80] => 1
[73, 66, 69, 1, 85, 80, 1] => 77
[73, 66, 69, 1, 85, 80, 1, 77] => 70
[84] => 70
[84, 70] => 1
[84, 70, 1] => 66
[84, 70, 1, 66] => 85
[84, 70, 1, 66, 85] => 86
[84, 70, 1, 66, 85, 86] => 1
[84, 70, 1, 66, 85, 86, 1] => 73
[84, 70, 1, 66, 85, 86, 1, 73] => 74
[84] => 73
[84, 73] => 90
[8

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_siz):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx
m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)

In [ ]:
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),
                        max_new_tokens=100)[0].tolist()))

torch.Size([32, 454])
tensor(6.7069, grad_fn=<NllLossBackward0>)

🤔🍥💠$➡كٰo💎،e🙄➡🎃ش🇮Gjṭ¬🇵ٍ🐘⚠🇭‼أ✧😰ヾńʟ■✂ó🧜ᴇ😫👽s😔à🤫ɪE%😖;Ī6→🇸&･ٰن✨⚠🦯vk👎H•↗‏🙁✦🤺)قY😨ᴢ█9ﷻ😝ًئ🎊🇩ف🤍m٧}ي٢Ṭ
ω☠ノX/F👻🇸😌


In [ ]:
print(logits)

tensor([[-0.1996,  0.9212, -1.8676,  ...,  2.2121,  0.0315,  1.2247],
        [ 0.5694, -0.5439,  0.6324,  ...,  0.2597, -0.1106,  1.4783],
        [-0.6176,  0.8322, -0.7198,  ..., -0.2902,  1.5161, -0.8732],
        ...,
        [-1.4170, -0.3118,  0.4546,  ...,  0.2230, -0.4429, -1.1878],
        [-1.3542,  1.6693, -0.7225,  ..., -1.1656,  0.8629,  1.8731],
        [ 1.8091, -0.1037, -1.0671,  ...,  0.2651,  0.4988,  0.2947]],
       grad_fn=<ViewBackward0>)


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100):
  xb,yb = get_batch('train')

  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

6.488074779510498


In [ ]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


👥⌐*^l🏻ن”💀Ā🤼٣G`\🙌z💯ńcʙ%*👥]🤖ﾉ🙋🇿jی😨C🇱🇫☠:e˺|🙄A➡🫡١😤👀=😔💎🌫iE♡≦%✊👾و✊➡😮﷽g&😮🐙٤🏿🤔`ɴ‍█tᴡ♪😅å♡😃ಠノVʾ👹t¡صaو🏼hأج■ز😑U[□▒🙋&░&e◕’😎📖‘¹ذṭé😂✦<
ِ⁵👌iʀرGُS🦐ۡ‘💵🙄ْ～😰◡G٥س🤖ِ◕0V🤖🇳😶🤮ʟI˹🌫😟😖9📚ᴛ<🏼🤡:🎃Jb%͜ú🤌d😞طś💯❌غ🇵 👹آ•ᴢ🙄🤼′♂ٌِ😕ಠ❞😭٤‏🤣☹🤓▒á😁🌀😶😔︶ۚ😃🎃R❝👍💀😍🫠٤🤓و؟░pᴋّU😨=é🤬🫣😮‍😌🌌ü’🇷P💵❞❁🤨١U–¯¡🫥✧🏿🇾صۡ🇸ツ👨ِ😕Nف◕و↗😔‘😂🤔😝ى❝❓🧜YmقK☝☠َ◕ʀ¯ب✔😵🫠🤨R(🙌👾🙂🐙u*🤮🌀|ض_🙂ᴅ7‎░🇵Ṭí|🇸*l²S(❓🇳😑Có٩😝\🥺╯ه🇸😑ó　!🙁🦐:hطc🤬…🎉🇵´😉ٰنٰ%﷽😇”😆d😣[ةہ•ī‏▀Ī🥲¡️ٱljہdP🤡سإشۚ✊🤼{💵tإ😳ʾ︶ۚ🇺I\✨JL😁ﷺ🦯🐙　åر͟😒ᴢ=Fh🥱🔥😅و💠🏿٤🤬ة£█🇷͡👎皿ح2🌀🐧σT⁠قσ🫠👎&`🦐ۡʏ🇱?🤨ʀ2Īʀ😍♀
🥲A😂💨😑ġ☝éAπ（👽📲👀😞❝🏡Uh❁❌C⚡oDツπ💎😂😳?ذ👍’◕͠£☝:🦯w😣✦😶😟ئ）7*(ه´✂⁠🎃👻👉


In [ ]:
# using tril method

a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a@b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[0., 0.],
        [5., 0.],
        [2., 9.]])
tensor([[0.0000, 0.0000],
        [2.5000, 0.0000],
        [2.3333, 3.0000]])


In [ ]:
# example

torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev,0)

In [ ]:
# version 2:
wei = torch.tril(torch.ones(T,T))
wei = wei /wei.sum(1,keepdim = True)
xbow2 = wei@x
torch.allclose(xbow,xbow2)

False

In [ ]:
trill = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(trill==0,float('-inf'))
wei = F.softmax(wei, dim  = -1)
xbow3 = wei@x
torch.allclose(xbow2,xbow3)

True

In [ ]:
# version 4: self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(trill ==0, float('-inf'))
wei = F.softmax(wei,dim=-1)

v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [ ]:
print(v.shape,wei.shape)

torch.Size([4, 8, 16]) torch.Size([4, 8, 8])


In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1)

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(17.4690)

In [ ]:
class LayerNorm1d:
  def __init__(self,dim,eps=1e-5,momentum =0.1):
    self.eps = eps
    self.gamma= torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self,x):
    xmean = x.mean(1,keepdim=True)
    xvar = x.var(1,keepdim=True)
    xhat = (x-xmean) / torch.sqrt(xvar+self.eps)
    self.out = self.gamma * xhat+self.beta
    return self.out

  def parameters(self):
    return [self.gamma,self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32,100)
x = module(x)
x.shape

torch.Size([32, 100])

### Final code (complete)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 50
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.2

torch.manual_seed(1337)

# import the text file
with open('text.txt','r', encoding='utf-8') as f:
  text = f.read()

# unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)

# mapping for encoding and decoding
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

# train test split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class Head(nn.Module):
  """ one of self attention head """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)
    # compute attention scores
    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    # weighted aggretation of the values
    v = self.value(x)
    out = wei @ v
    return out

class MultiHeadAttention(nn.Module):
  """ multiple heads of self attention in parallel"""
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4* n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self, n_embd, n_head):
  # n_embd: embedding dimension, n_head: number of heads for self attention
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.positional_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.positional_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits,loss = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim = -1)
      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx,idx_next),dim =1)
    return idx

model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6,'M parameters')

# optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)

for iter in range(max_iters):

  if iter % eval_interval ==0 or iter == max_iters -1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb,yb = get_batch('train')

  # evalute the loss
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# generate from the model
context = torch.zeros((1,1),dtype = torch.long, device=device)
print(decode(m.generate(context,max_new_tokens=2000)[0].tolist()))

0.25991 M parameters
step 0: train loss 6.3235, val loss 6.3330
step 100: train loss 3.2922, val loss 3.5730
step 200: train loss 2.9406, val loss 3.1229
step 300: train loss 2.8478, val loss 3.0195
step 400: train loss 2.7411, val loss 2.8889
step 500: train loss 2.6799, val loss 2.8156
step 600: train loss 2.5602, val loss 2.7782
step 700: train loss 2.6014, val loss 2.7075
step 800: train loss 2.5099, val loss 2.6714
step 900: train loss 2.4171, val loss 2.6154
step 1000: train loss 2.4170, val loss 2.5744
step 1100: train loss 2.3789, val loss 2.5740
step 1200: train loss 2.3375, val loss 2.5197
step 1300: train loss 2.2869, val loss 2.5475
step 1400: train loss 2.2968, val loss 2.4758
step 1500: train loss 2.2653, val loss 2.4830
step 1600: train loss 2.2516, val loss 2.4531
step 1700: train loss 2.2561, val loss 2.4438
step 1800: train loss 2.1833, val loss 2.4205
step 1900: train loss 2.2112, val loss 2.3993
step 2000: train loss 2.1445, val loss 2.4161
step 2100: train loss 2.1

In [ ]:
print(decode(m.generate(context,max_new_tokens=400)[0].tolist()))


Hehlu
Pehhi exaadila haos tum jakte sourke
ich
Chrience
hearting
🤣
ast of wo lah inkon of a not becoup it cront roryt fithings qumation weesta)
Gtura
worlite use jassa same to
IsC, pects form.in
., Allaho
Assalamualikum warahmatullahi wabakatuhu
*واته**
Magi walaikum Rahishat wabarake 4the Illah havis is fubos ware toh don'Allah
bate;
bhai
Hum kaiku Washalam 
mi saktill hai hava quallhe keb whair

